# 文段句格式语料转wikipedia格式jsonlines

原始预料的格式是 JSON Lines，每一行是一篇文章，其内容是二维数组，第一个维度是段落，第二个维度是句子

## 代码准备

### Imporings

In [ ]:
import json
import math
import os
import sys
from contextlib import ExitStack
from datetime import timedelta
from functools import partial
from glob import glob, iglob
from itertools import chain, cycle, islice
from multiprocessing import Pool
from time import time

import numpy as np
import sentencepiece as spm
from tqdm.auto import tqdm


### Constants

In [ ]:
SEQ_LENGTH = 1024
MIN_CTX_LEN = 128

SPM_MODEL = '../data/spm/gpt2_huamei_corpus_bpe_32k_v2.model'


SP = spm.SentencePieceProcessor()
SP.load(SPM_MODEL)

### Functions

In [2]:
def text_files_line_iterator(paths):
    return chain.from_iterable(
        open(path)
        for path
        in tqdm(paths, '[files     ]', unit='file')
    )


def single_text_file_line_count(path, show_progress_bar=False):
    with open(path) as fd:
        iterable = tqdm(fd) if show_progress_bar else fd
        return sum(1 for _ in iterable)


def text_files_line_count(paths):
    try:
        total = len(paths)
    except (AttributeError, TypeError):
        total = None
    with Pool() as pool:
        it = pool.imap_unordered(
            single_text_file_line_count,
            tqdm(paths, '[files     ]', unit='file')
        )
        return sum(c for c in tqdm(it, '[files     ]', unit='file', total=total))


def proc_line(line):
    result = []
    line = line.strip()
    if not line:
        return result
    paragraphs = json.loads(line)
    text = ''
    n_text = 0
    for sentence in chain.from_iterable(paragraphs):
        sentence = sentence.strip()
        if not sentence:
            continue
        n_sentence = len(SP.encode_as_ids(sentence))
        if n_text + n_sentence < SEQ_LENGTH + MIN_CTX_LEN // 2:
            text += sentence
            n_text += n_sentence
        else:
            result.append({'text': text, 'length': n_text})
            text = sentence
            n_text = n_sentence
    if n_text:
        result.append({'text': text, 'length': n_text})
    return result
    

## 语料文件

### 输入文件

In [ ]:
INPUT_FILES = [
    path
    for path in tqdm(iglob(
        "/nfs/server01_public/data/gpt2/output/gpt2_huamei_corpus.json.8g/**/*.*",
        recursive=True
    ))
    if os.path.isfile(path) and os.path.splitext(path)[1].lower() in ('.jsonl', '.jsonlines', 'json', 'jsonline')
]
print(f'源语料文件数：{len(INPUT_FILES):,d}')

total_lines = text_files_line_count(INPUT_FILES)
print(f'源语料行数：{total_lines:,d}')

### 输出文件

In [ ]:
OUTPUT_FILE = '../data/gpt2_huamei_corpus_10g-1k.jsonl'

## 执行

In [ ]:
lines_iterator = text_files_line_iterator(INPUT_FILES)
n_samples = 0
n_discard = 0

with Pool() as pool, \
     open(OUTPUT_FILE, 'w') as fp:
    it = pool.imap_unordered(
        proc_line,
        tqdm(lines_iterator, 'process', total=total_lines),
        chunksize=512
    )
    for result in tqdm(it, 'save', total=total_lines):
        for d in result:
            if d['length'] < MIN_CTX_LEN:
                n_discard += 1
                continue
            s = json.dumps(d, ensure_ascii=False)
            n_samples += 1
            print(s, file=fp)

print(f'得到语料样本数：{n_samples:,d}')
print(f'抛弃语料样本数：{n_discard:,d}')

### 查看输出文件

In [ ]:
x = !wc -l {OUTPUT_FILE}
n_samples = int(x[0].split()[0])

print(x[0])

!du -h {OUTPUT_FILE}

## 长度统计

见：  <http://holoviews.org/user_guide/Large_Data.html>

In [ ]:
import json

import pandas as pd
import holoviews as hv
import hvplot.pandas  # noqa

import datashader as ds
import datashader.transfer_functions as tf

from holoviews.operation.datashader import datashade, shade, spread, dynspread, rasterize, spread
from tqdm.auto import tqdm

# hv.extension('bokeh')

def iter_corpus_length():
    with open(OUTPUT_FILE) as fp:
        for line in tqdm(fp, total=n_samples):
            data = json.loads(line)
            yield {'length': data['length']}


In [ ]:
df = pd.DataFrame(iter_corpus_length())

In [ ]:
df.describe()

In [ ]:
points = hv.Points(df, ['index', 'length'])
spread(datashade(points))